Importanto bibliotecas necessárias

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy import stats

c:\Users\gabri\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Introdução


**Contextualização**

O objetivo do presente projeto de Ciência de Dados é explorar e analisar indicadores de fluxo da Educação Superior a partir de uma base de dados governamentais. O projeto tem a finalidade de prever o comportamento de uma variável principal em relação a diversas outras variáveis que podem influenciar seu desempenho. Na linguagem da estatística e da Ciência de Dados, a variável principal é comumente chamada de "Target", enquanto as demais variáveis que a influenciam são conhecidas como "Features".

**Tema e Objetivo**

O tema central deste projeto gira em torno da Educação Superior, sendo o objetivo principal responder a seguinte pergunta:

    É possível prever o número de desistências em cursos de graduação com base em indicadores específicos?

Para atingir esse objetivo, foi escolhida uma base de dados do governo nacional que contém informações abrangentes sobre instituições de ensino, cursos de graduação e seus respectivos indicadores. Utilizaremos esses dados para construir um modelo preditivo que permita prever a quantidade de desistências em cursos de graduação. A variável principal (ou "Target") é a "Quantidade de Desistência no Curso no ano de referência".

**Base de Dados**

A fonte de dados é uma base governamental, composta por microdados, que fornece informações detalhadas sobre instituições de ensino, cursos de graduação e outros dados acadêmicos. Essa base contém colunas que descrevem a instituição, o curso, a região geográfica, a unidade federativa, o grau acadêmico, bem como diversas métricas educacionais, como a quantidade de ingressantes, a quantidade de concluintes, a taxa de permanência e a taxa de conclusão, entre outras.

A base de dados pode ser obtida através do link abaixo, selecionando os anos 2018-2022.
https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/indicadores-educacionais/indicadores-de-fluxo-da-educacao-superior 

Uma amostra dessa base de dados pode ser vista na seção abaixo:

In [2]:
dados = pd.read_excel('indicadores_trajetoria_educacao_superior_2018_2022.xlsx')
dados.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'indicadores_trajetoria_educacao_superior_2018_2022.xlsx'

**Dicionário da base de dados**

- Código da Instituição: valor numérico referente à instituição
- Nome da Instituição: nome da instituição de ensino a que se referem os dados
- Categoria Administrativa: código da cateoria administrativa da instituição de ensino no último ano de análise
    1. Pública Federal
    2. Pública Estadual
    3. Pública Municipal
    4. Privada com fins lucrativos
    5. Privada sem fins lucrativos
    7. Especial
- Organização Acadêmica: código da organização acadêmica da instituição de ensino
    1. Universidade
    2. Centro Universitário
    3. Faculdade
    4. Instituto Federal de Educação, Ciência e Tecnologia                                
    5. Centro Federal de Educação Tecnológica
- Código do Curso de Graduação: código único de identificação do curso gerado pelo E-MEC
- Nome do Curso de Graduação: nome do curso com a informação do último ano de análise
- Código da Região Geográfica do Curso: código da Região Geográfica do local de oferta do curso gerado pelo E-MEC no último ano de análise
    1. Região Norte
    2. Região Nordeste
    3. Região Sudeste
    4. Região Sul
    5. Região Centro-Oeste
- Código da Unidade Federativa do Curso: código do estado da federação do local de oferta do curso
- Código do Município do Curso: código do município do local de oferta do curso gerado 
- Grau Acadêmico: grau conferido ao diplomado pelo curso
    1. Bacharelado
    2. Licenciatura
    3. Tecnológico
- Modalidade de Ensino: modalidade do curso no último ano de análise
    1. Presencial
    2. Curso a distância
- Código da área do Curso segundo a classificação CINE BRASIL: Código da área geral conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco.
- Nome da área do Curso segundo a classificação CINE BRASIL: Nome da área geral conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco.
- Código da Grande Área do Curso segundo a classificação CINE BRASIL: Código de identificação do curso, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco.
- Nome da Grande Área do Curso segundo a classificação CINE BRASIL: Nome de identificação do curso, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco.
- Ano de Ingresso: Ano de ingresso do aluno no curso.
- Ano de Referência: Ano de referência do vínculo do ingressante.
- Prazo de Integralização em Anos: Prazo mínimo de integralização de curso de graduação em número de anos.
- Ano de Integralização do Curso: Ano previsto de integralização do aluno ao curso.
- Prazo de Acompanhamento do Curso em anos: Prazo máximo de integralização de curso de graduação em número de anos.
- Ano Máximo de Acompanhamento do Curso: Ano máximo de acompanhamento da situação de vínculo do aluno.
- Quantidade de Ingressantes no Curso: Número de ingressantes do curso no ano de ingresso da coorte.
- Quantidade de Permanência no Curso no ano de referência: Número de estudantes que permaneceram no curso de graduação no ano de referência da análise.
- Quantidade de Concluintes no Curso no ano de referência: Número de estudantes que concluíram o curso de graduação no ano de referência da análise.
- Quantidade de Desistência no Curso no ano de referência: Número de estudantes que desistiram do curso de graduação no ano de referência da análise.
- Quantidade de Falecimentos no Curso no ano de referência: Número de estudantes que faleceram no ano de referência da análise.
- Taxa de Permanência - TAP: Percentual de ingressantes que estão com vínculo ativo no curso no ano de referência.
- Taxa de Conclusão Acumulada - TCA: Percentual de ingressantes que concluíram o curso até o ano de referência.
- Taxa de Desistência Acumulada - TDA: Percentual de ingressantes que desistiram do curso até o ano de referência.
- Taxa de Conclusão Anual - TCAN: Percentual de ingressantes que concluíram o curso no ano de referência.
- Taxa de Desistência Anual - TADA: Percentual de ingressantes que desistiram do curso no ano de referência.

**Features considerados**

A partir da análise das informações contidas na base de dados, é possível identificar que nem todas as colunas dessa base possuem informações relevantes para o objetivo de prever o número de desistências em cursos de graduação. Algumas colunas possuem informações redundantes, e outras simplesmente representam códigos que fogem do interesse da análise. Portanto, foi selecionado um conjunto específico de colunas que podem ser mais pertinentes para a construção do modelo preditivo. Essas colunas, que representarão as "Features" ou variáveis independentes, são as seguintes:

    Nome da Instituição
    Categoria Administrativa
    Organização Acadêmica
    Código da Unidade Federativa do Curso
    Código do Município do Curso
    Grau Acadêmico
    Modalidade de Ensino
    Nome da área do Curso segundo a classificação CINE BRASIL
    Ano de Ingresso
    Ano de Referência
    Quantidade de Ingressantes no Curso
    Quantidade de Permanência no Curso no ano de referência
    Quantidade de Concluintes no Curso no ano de referência
    
    
**Variável de Destino (Target)**

Como o objetivo principal é prever o número de desistências em cursos de graduação, a variável de destino ou "Target" que será previsto é a "Quantidade de Desistência no Curso no ano de referência". Esta variável é fundamental, pois representa o número de alunos que abandonaram o curso durante o ano de referência, e o objetivo é entender os fatores que podem influenciar essa taxa de desistência.

A seleção cuidadosa das colunas de interesse e a escolha da variável de destino são passos essenciais para a construção de um modelo preditivo preciso e informativo. Com essas informações será possível realizar a análise e construção de modelos que permitirão prever e compreender as desistências em cursos de graduação.




**Estrutura do Projeto**

Serão exploradas técnicas de regressão para a previsão da variável quantitativa "Quantidade de Desistência no Curso". A qualidade da comunicação e apresentação dos resultados também será uma prioridade, assegurando que os resultados sejam compreensíveis para um público amplo.

Este projeto tem como objetivo fornecer previsões robustas e ir além de uma simples análise exploratória de dados. Espera-se que a análise e modelos desenvolvidos ajudem a entender e prever as desistências em cursos de graduação, o que pode ter implicações importantes na melhoria da Educação Superior.

# Minerando Dados e Características do Dataset

# Modelos de Predição

# Processo e Estatísticas de Validação

# Conclusão 

# Referências Bibliográficas